在上一节中，我们简单介绍了nanoGPT的基本方式，但是我们也能看出这个GPT过于简陋，其生成效果急需进一步的提高


### 首先是编码，嵌入方式的改进
- 之前我们采用的是简单的一一对应的方式
    - 对于只有26个大写和26个小写的英文字符来说，这样似乎还算合理，因为只是把50多个或者60多个字符按照顺序去编码为对应的数字而已

- 例如，OpenAI 在之前的GPT-2，GPT-3，GPT-4系列中使用的是其
发布的 tiktoken 库，而 Google 也有其自己的分词/编码工具 SentencePiece，他们只是
不同的方式，但做的都是将“完整的句子转化成整数向量编码”这样的一件事情。例
如，我们现在可以利用 tiktoken 库来十分方便地调用 GPT-2 中所训练的 tokenizer，从
而实现编解码过程

In [1]:
# Way2
import tiktoken
enc = tiktoken.get_encoding("cl100k_base")
# enc = tiktoken.get_encoding("gpt2")
print(enc.encode("我是孙悟空"))
print(enc.decode(enc.encode("我是孙悟空")))

[37046, 21043, 10890, 247, 162, 224, 253, 35894]
我是孙悟空


之前经过encoder之后的数量不会改变

In [2]:
# 还是跟之前一样，我们先进行文本的读取
with open('../data/Xiyou.txt', 'r', encoding='utf-8') as f:
    text = f.read()

n = int(0.5*len(text)) # 前90%都是训练集，后10%都是测试集
text = text[:n]

  # 对文本进行编码
len(enc.encode(text))   # 获取编码之后的长度

489540

接下来我们检查一下这样是否work、以及这样是否可以提升性能

In [3]:
import torch
import torch.nn.functional as F
import torch.nn as nn


# 开始划分训练集和测试集
code = enc.encode(text)
data = torch.tensor(code, dtype=torch.long)   # Way 1 

vocab_size = len(code)
n = int(0.9*len(data)) # 前90%都是训练集，后10%都是测试集
train_data = data[:n]
val_data = data[n:]

# 进行参数的设置
device = 'cpu'  # 模型运行的设备
block_size = 16  # 每个单元的最大长度
batch_size = 32 # 同时运行的批次大小
learning_rate = 0.3
max_iters = 1000
eval_interval = 300  # 对当前模型的运行结果进行评估的epoch数量
eval_iters = 200

# 每次从数据集中获取x和y
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

class BLM(nn.Module):
    def __init__(self,vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size,vocab_size)
    
    def forward(self,idx,targets = None):
        # 这里的self，就是我们之前的x，target就是之前的y
        logits = self.token_embedding_table(idx) # (B,T)  -> (B,T,C)  # 这里我们通过Embedding操作直接得到预测分数
        # 这里的预测分数过程与二分类或者多分类的分数是大致相同的

        
        if targets is None:
            loss = None
        else:   
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)    # 这里我们调整一下形状，以符合torch的交叉熵损失函数对于输入的变量的要求
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        '''
        idx 是现在的输入的(B, T) array of indices in the current context
        max_new_tokens 是产生的最大的tokens数量
        '''

        for _ in range(max_new_tokens):
            # 得到预测的结果
            logits, loss = self(idx)
            
            # 只关注最后一个的预测
            logits = logits[:, -1, :] # becomes (B, C)
            # 对概率值应用softmax
            probs = F.softmax(logits, dim=-1) # (B, C)
            # 对input的每一行做n_samples次取值，输出的张量是每一次取值时input张量对应行的下标，也即找到概率值输出最大的下标，也对应着最大的编码
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # 将新产生的编码加入到之前的编码中，形成新的编码
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx 

In [4]:
# 创建模型
model = BLM(vocab_size)
m = model.to(device)

: 

In [ ]:
# 简单的写下
optimizer = torch.optim.AdamW(m.parameters(), lr=learning_rate)
for steps in range(1000): # 随着迭代次数的增长，获得的效果会不断变好

    xb, yb = get_batch('train')

    logits, loss = m(xb, yb)  # 用于评估损失
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
    print(epoch)
    print(loss.item())

print(loss.item())

optimizer.step()

In [ ]:
print(enc.decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=500)[0].tolist()))